In [2]:
import numpy as np
from scipy.special import jv
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# TRANSMITTER_POWER_W = 12.0 # High power mode
TRANSMITTER_POWER_W = 2.0 # Low power mode
TX_ANT_GAIN_DBI = -1.25
# TX_ANT_GAIN_DBI = 3.30
# TX_ANT_GAIN_DBI = 0.00

TX_CABLE_LOSS_DB = 2.75 # Good
FREQ_MHZ = 2250 # Good
L_ATM = 0.19 # Good
L_POL = 0.22 # Good
RX_GT_DB_K = 33.63
K_BOLTZMANN_DBW = -228.599167  # 10*log10(1.380649e-23)
RX_SYSTEM_LOSS_DB = 0.6
CARRIER_LOOP_BW_HZ = 45.0      # 2B0 (Carrier Loop Noise Bandwidth)
DATA_RATE_BPS= (np.array([1024, 512, 256, 128, 64])) * 1000
SYMBOL_RATE_SPS = 1187.84 * 1000 # Specific to 1024 kbps mode
MOD_INDEX_RAD = 1.25           # theta_1 (Data)
RNG_MOD_INDEX = 0.176          # theta_2 (Ranging)
CMD_MOD_INDEX = 0.236          # theta_3 (Command turn-around)
REQUIRED_CARRIER_SNR = 10.0
REQUIRED_EBNO = 2.55
DSN_ANT_GAIN_DBI = 55.93       # DSS27 Ground Gain
DSN_MISC_LOSS_DB = 0.10        # Coupling/Misc loss in Ground Rx


def calculate_margins(altitudes, data_rate):
    # a. EIRP and Propagation
    eirp = (10 * np.log10(TRANSMITTER_POWER_W)) - TX_CABLE_LOSS_DB + TX_ANT_GAIN_DBI
    path_loss = 32.45 + (20 * (np.log10(FREQ_MHZ) + np.log10(altitudes)))
    rip_dbw = eirp - path_loss - L_ATM - L_POL

    # Carrier Suppression: Pc/Pt = cos^2(t1) * J0^2(t2) * J0^2(t3)
    sup_c = 10 * (np.log10(np.cos(MOD_INDEX_RAD)**2) + 
                  np.log10(jv(0, RNG_MOD_INDEX)**2) + 
                  np.log10(jv(0, CMD_MOD_INDEX)**2))

    # TLM Suppression: Pd/Pt = sin^2(t1) * J0^2(t2) * J0^2(t3)
    sup_tlm = 10 * (np.log10(np.sin(MOD_INDEX_RAD)**2) +
                  np.log10(jv(0, RNG_MOD_INDEX)**2) +
                  np.log10(jv(0, CMD_MOD_INDEX)**2))

    # Eb/No Margin
    recieved_pr_no= rip_dbw + 34.83 - K_BOLTZMANN_DBW
    playback= 10 * np.log10(data_rate)
    pd_no = recieved_pr_no + sup_tlm
    eb_no_net = pd_no - RX_SYSTEM_LOSS_DB - playback
    eb_no_margin = eb_no_net - REQUIRED_EBNO

    # e. Ground AGC (Received Carrier Power in dBm)
    dsn_rcvr_agc_dbm = rip_dbw + 30 + DSN_ANT_GAIN_DBI - DSN_MISC_LOSS_DB + sup_c

    return eb_no_margin, dsn_rcvr_agc_dbm


def add_plot_data(fig, data_rate, row):
    "build the RF link margin plot"

    altitudes_km = np.linspace(500, 160000, 2000)

    eb_no_margin, dsn_rcvr_agc_dbm = calculate_margins(altitudes_km, data_rate)

    # Ref max altitude
    ref_alt = 140000
    eb_no_margin_ref, dsn_rcvr_agc_dbm_ref = calculate_margins(ref_alt, data_rate)
    print(f"Data Rate: {data_rate/1000} kbps at {ref_alt} km -> Eb/No Margin: "
          f"{eb_no_margin_ref:.2f} dB, DSN RCVR AGC: {dsn_rcvr_agc_dbm_ref:.2f} dBm")

    # Add Eb/No values
    fig.add_trace(go.Scatter(x=altitudes_km, y=eb_no_margin,
                            name= f'Eb/No Margin (dB) ({data_rate/1000})', 
                            line=dict(color='green', width=2)), row=row, col=1)

    fig.add_trace(go.Scatter(x=altitudes_km, y=dsn_rcvr_agc_dbm, name= 'DSN RCVR AGC /w Ranging (dBm)', 
                            line=dict(color='orange', width=2, dash='dash')), row=row, col=1,
                            secondary_y= True)

    # Vertical Reference Line at 140,000 km
    fig.add_vline(x=ref_alt, line_dash="dash", line_color="red", opacity=0.7)

    # Annotations for reference point
    fig.add_annotation(x=ref_alt, y=eb_no_margin_ref, text=f"{eb_no_margin_ref:.2f} dB",
                    showarrow=True, row= row, col= 1, font= dict(color= "#FFFFFF"))
    fig.add_annotation(x=ref_alt, y=dsn_rcvr_agc_dbm_ref, text=f"{dsn_rcvr_agc_dbm_ref:.2f} dBm",
                    showarrow=True, row= row, col= 1, font= dict(color= "#FFFFFF"),
                    secondary_y= True)

    return fig


def format_plot(fig, row):
    "Final formatting of the plot object after generation"
        # Layout adjustments
    fig.update_layout(
        title= dict(text= f"AXAF Link Budget Analysis (34M DSS27, {[f'{x / 1000} kbps' for x in DATA_RATE_BPS]})",
                    font= dict(color= "#FFFFFF")),
        paper_bgcolor= "#202020",
        plot_bgcolor= "#000000",
        height= 2000, width= 1250,
        showlegend= False
        )

    # X-Axis
    getattr(fig.layout, f"{'xaxis' if {row - 1} == 0 else f'xaxis{row}'}").title.text= "Altitude (km)"
    getattr(fig.layout, f"{'xaxis' if {row - 1} == 0 else f'xaxis{row}'}").color= "#FFFFFF"

    # Y-Axis
    for y in range(4):
        if (row- 1) % 2 == 0:
            getattr(fig.layout, f"{'yaxis' if row== 1 else f'yaxis{row+ (y*2)}'}").title.text= "Eb/No Margin (dB)"
            getattr(fig.layout, f"{'yaxis' if row== 1 else f'yaxis{row+ (y*2)}'}").title.font.color= "#FFFFFF"
            getattr(fig.layout, f"{'yaxis' if row== 1 else f'yaxis{row+ (y*2)}'}").color= "#FFFFFF"
            getattr(fig.layout, f"{'yaxis' if row== 1 else f'yaxis{row+ (y*2)}'}").range= [-5, 50]
        else:
            getattr(fig.layout, f"yaxis{row+ (y*2)}").title.text= "DSN RCVR AGC /w Ranging (dBm)"
            getattr(fig.layout, f"yaxis{row+ (y*2)}").title.font.color= "#FFFFFF"
            getattr(fig.layout, f"yaxis{row+ (y*2)}").color= "#FFFFFF"
            getattr(fig.layout, f"yaxis{row+ (y*2)}").range= [-170, -60]

    fig.layout.annotations[row- 1].update(font= dict(color= "#FFFFFF"))


def main():
    # Init plotly object
    fig = make_subplots(rows= (len(DATA_RATE_BPS) + 1), cols= 1, shared_xaxes= True,
                        vertical_spacing= 0.04,
                        subplot_titles= [f"{x / 1000} kbps" for x in DATA_RATE_BPS],
                        specs= [[{"secondary_y": True}] for x in range(len(DATA_RATE_BPS) + 1)])

    for row, (data_rate) in enumerate(DATA_RATE_BPS):
        add_plot_data(fig, data_rate, row + 1)
        format_plot(fig, row + 1)

    fig.update_xaxes(
            showspikes=True, spikemode='across', spikesnap='cursor',
            spikethickness=1, spikecolor="white", spikedash="solid",
            matches='x')

    fig.update_layout(
        hovermode='x', spikedistance=-1, hoverdistance=-1,
        paper_bgcolor="#202020", plot_bgcolor="#000000",
        height=2000,  width=1250, showlegend=False)

    fig.show()
    # fig.write_html("AXAF RF Link Budget.html")


main()


Data Rate: 1024.0 kbps at 140000 km -> Eb/No Margin: -4.28 dB, DSN RCVR AGC: -128.20 dBm
Data Rate: 512.0 kbps at 140000 km -> Eb/No Margin: -1.27 dB, DSN RCVR AGC: -128.20 dBm
Data Rate: 256.0 kbps at 140000 km -> Eb/No Margin: 1.74 dB, DSN RCVR AGC: -128.20 dBm
Data Rate: 128.0 kbps at 140000 km -> Eb/No Margin: 4.75 dB, DSN RCVR AGC: -128.20 dBm
Data Rate: 64.0 kbps at 140000 km -> Eb/No Margin: 7.76 dB, DSN RCVR AGC: -128.20 dBm


In [88]:
%reset -f